Tabs:
    LU TBM
    Hu-LU Material Types
    Pass Counts
    Pass Quan
    Pass Time
    Nodes
    Paths

In [1]:
import pandas as pd
from itertools import product
from scipy.stats import norm
import numpy as np
from operator import attrgetter
import openpyxl

In [2]:
fileName = '../data/demo-files/ModelSettings.xlsx'
minCycles = 500

In [3]:
huModelsDF = pd.read_excel(fileName, sheet_name='HU Types', index_col=0, usecols='A:B')
luModelsDF = pd.read_excel(fileName, sheet_name='LU Types', index_col=0, usecols='A:B')
materialsDF = pd.read_excel(fileName, sheet_name='Material Types', index_col=0, usecols='A:D')
dumpsDF = pd.read_excel(fileName, sheet_name='Dump Locations', index_col=0, usecols='A:B')

In [4]:
def create_distribution(eq_type, dist_type, mean, sigma = 0, cycle_count = 500, eq_typeID = -1, bin_count = 10):
    df = pd.DataFrame()

    df['Rate'] = pd.cut(norm.rvs(loc=mean, scale=sigma, size=cycle_count),
                                   bins=bin_count)
    df = df.Rate.value_counts().reset_index().rename(columns={'index':'Time'}).sort_values(by='Time')
    df = df.reset_index(drop=True)
    df['Time'] = df['Time'].map(attrgetter('left'))
    df['EQ Group'] = eq_type
    df['EQ Type ID'] = eq_typeID
    df['Delay Type'] = dist_type
                        
    df = df[['EQ Group', 'EQ Type ID', 'Delay Type', 'Time', 'Rate']]
                        
    return df

In [5]:
def dist_bytype(eq_type, huDF, dists):
    df = pd.DataFrame()
    
    for dist in dists:
        for index, row in huDF.iterrows():
            dist_mean = dist + '-mean'
            dist_sigma = dist + '-sigma'
            
            df = pd.concat([df,
                            create_distribution(eq_type = eq_type,
                                                dist_type = dist,
                                                mean = row[dist_mean],
                                                sigma = row[dist_sigma],
                                                cycle_count = minCycles,
                                                eq_typeID = index)])
    
    return df

In [6]:
huModelsDF

,Type Name
Type ID,
0,CAT797F
1,CAT794AC
2,DRS830E
3,DRS930E
4,KOM980E


In [7]:
# Delays by HU type
huModelsDF['TBD-mean'] = [10, 15, 20, 25, 30]
huModelsDF['TBD-sigma'] = 1

huModelsDF['TTD-mean'] = [10, 15, 20, 25, 30]
huModelsDF['TTD-sigma'] = 2

# Standbys by HU type
huModelsDF['TBS-mean'] = [10, 15, 20, 25, 30]
huModelsDF['TBS-sigma'] = 2

huModelsDF['TTS-mean'] = [10, 15, 20, 25, 30]
huModelsDF['TTS-sigma'] = 3

# Failures by HU type
huModelsDF['TBF-mean'] = [10, 15, 20, 25, 30]
huModelsDF['TBF-sigma'] = 3

huModelsDF['TTF-mean'] = [10, 15, 20, 25, 30]
huModelsDF['TTF-sigma'] = 4

In [8]:
huDelay = dist_bytype('HU', huModelsDF, ['TBD', 'TTD'])
huStandby = dist_bytype('HU', huModelsDF, ['TBS', 'TTS'])
huFailure = dist_bytype('HU', huModelsDF, ['TBF', 'TTF'])

In [9]:
# Variables for ALL HU Types
huShiftChange_mean = 20
huShiftChange_sigma= 5

huErgo20_mean = 20
huErgo20_sigma = 2.5
 
huLunch15_mean = 15
huLunch15_sigma = 2.5

huLunch20_mean = 20
huLunch20_sigma = 2.5

In [10]:
# Compute Shift Change Distribtion
huShiftChange = pd.DataFrame()
huShiftChange = create_distribution(eq_type = 'HU',
                                    dist_type = 'Shift Change',
                                    mean = huShiftChange_mean,
                                    sigma = huShiftChange_sigma,
                                    cycle_count = minCycles*len(huModelsDF))

# Compute Ergo 20 Distribution
huErgo20 = pd.DataFrame()
huErgo20 = create_distribution(eq_type = 'HU',
                               dist_type = 'Ergo 20',
                               mean = huErgo20_mean,
                               sigma = huErgo20_sigma,
                               cycle_count = minCycles*len(huModelsDF))


# Compute Lunch 15 Distribution
huLunch15 = pd.DataFrame()
huLunch15 = create_distribution(eq_type = 'HU',
                                dist_type = 'Lunch 15',
                                mean = huLunch15_mean,
                                sigma = huLunch15_sigma,
                                cycle_count = minCycles*len(huModelsDF))

# Compute Lunch 20 Distribution
huLunch20 = pd.DataFrame()
huLunch20 = create_distribution(eq_type = 'HU',
                                dist_type = 'Lunch 20',
                                mean = huLunch20_mean,
                                sigma = huLunch20_sigma,
                                cycle_count = minCycles*len(huModelsDF))

In [11]:
# Combine all of the random HU delays
huRandomDelaysCombined = pd.concat([huDelay, huStandby, huFailure, huShiftChange, huErgo20, huLunch15, huLunch20])
huRandomDelaysCombined = huRandomDelaysCombined.reset_index(drop=True)
huRandomDelaysCombined.sample(10)

,EQ Group,EQ Type ID,Delay Type,Time,Rate
249,HU,4,TBF,36.425,5
318,HU,-1,Ergo 20,26.051,27
221,HU,2,TBF,12.354,16
178,HU,2,TTS,25.439,13
304,HU,-1,Shift Change,18.190,703
183,HU,3,TTS,20.779,64
257,HU,0,TTF,16.446,24
85,HU,3,TTD,25.373,102
43,HU,4,TBD,28.843,93
204,HU,0,TBF,8.156,111


In [12]:
luModelsDF

,Type Name
Type ID,
0,CAT994
1,CAT994H
2,KOMATSU WA1200
3,L-1850
4,L-1850 2ND
5,PC5500
6,PH4100
7,PH4100XPB
8,PH4100XPC


In [13]:
# Delays by LU type
luModelsDF['TBD-mean'] = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65]
luModelsDF['TBD-sigma'] = 1

luModelsDF['TTD-mean'] = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65]
luModelsDF['TTD-sigma'] = 2

# Standbys by LU type
luModelsDF['TBS-mean'] = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65]
luModelsDF['TBS-sigma'] = 2

luModelsDF['TTS-mean'] = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65]
luModelsDF['TTS-sigma'] = 3

# Failures by LU type
luModelsDF['TBF-mean'] = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65]
luModelsDF['TBF-sigma'] = 3

luModelsDF['TTF-mean'] = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65]
luModelsDF['TTF-sigma'] = 4

In [14]:
luDelay = dist_bytype('LU', luModelsDF, ['TBD', 'TTD'])
luStandby = dist_bytype('LU', luModelsDF, ['TBS', 'TTS'])
luFailure = dist_bytype('LU', luModelsDF, ['TBF', 'TTF'])

In [15]:
# Variables for ALL LU Types
luShiftChange_mean = 20
luShiftChange_sigma= 5

luErgo20_mean = 20
luErgo20_sigma = 2.5

luLunch15_mean = 15
luLunch15_sigma = 2.5

luLunch20_mean = 20
luLunch20_sigma = 2.5

In [16]:
# Compute Shift Change Distribtion
luShiftChange = pd.DataFrame()
luShiftChange = create_distribution(eq_type = 'LU',
                                    dist_type = 'Shift Change',
                                    mean = luShiftChange_mean,
                                    sigma = luShiftChange_sigma,
                                    cycle_count = minCycles*len(luModelsDF))

# Compute Ergo 20 Distribution
luErgo20 = pd.DataFrame()
luErgo20 = create_distribution(eq_type = 'LU',
                               dist_type = 'Ergo 20',
                               mean = luErgo20_mean,
                               sigma = luErgo20_sigma,
                               cycle_count = minCycles*len(luModelsDF))


# Compute Lunch 15 Distribution
luLunch15 = pd.DataFrame()
luLunch15 = create_distribution(eq_type = 'LU',
                                dist_type = 'Lunch 15',
                                mean = luLunch15_mean,
                                sigma = luLunch15_sigma,
                                cycle_count = minCycles*len(luModelsDF))

# Compute Lunch 20 Distribution
luLunch20 = pd.DataFrame()
luLunch20 = create_distribution(eq_type = 'LU',
                                dist_type = 'Lunch 20',
                                mean = luLunch20_mean,
                                sigma = luLunch20_sigma,
                                cycle_count = minCycles*len(luModelsDF))

In [17]:
# Combine all of the random HU delays
luRandomDelaysCombined = pd.concat([luDelay, luStandby, luFailure, luShiftChange, luErgo20, luLunch15, luLunch20])
luRandomDelaysCombined = luRandomDelaysCombined.reset_index(drop=True)
luRandomDelaysCombined.sample(10)

,EQ Group,EQ Type ID,Delay Type,Time,Rate
65,LU,6,TBD,40.369,102
299,LU,5,TBS,40.939,2
564,LU,8,TBF,48.214,129
100,LU,10,TBD,56.782,2
155,LU,3,TTD,24.880,96
207,LU,8,TTD,53.400,9
464,LU,10,TTS,58.383,132
188,LU,6,TTD,43.172,23
343,LU,10,TBS,57.743,98
655,LU,5,TTF,35.664,119


Dump Delays

In [18]:
dumpsDF

,Location Name
Location ID,
0,1 STK
1,5 STK
2,Breaker
3,EAST 1925
4,InpitEagle
5,InpitLM
6,InpitSwift
7,N2BF 1820
8,N2BF 1835


In [19]:
# Dump times per dump location in seconds
dumpsDF['time-mean'] = 1
dumpsDF['time-sigma'] = 0.5

In [20]:
dumpDelay = dist_bytype('DL', dumpsDF, ['time']).reset_index(drop=True)

In [21]:
# LU Time Between Movements


In [22]:
lu_df = luModelsDF.reset_index().rename(columns={'Type ID':'LU Type ID'})
hu_type_df = huModelsDF.reset_index().rename(columns={'Type Name':'HU_MODEL','Type ID':'HU Type ID'})
material_df = materialsDF.reset_index().rename(columns={'Type Name':'MATERIAL_DESCRIP','Type ID':'Material Type ID'})

spottimes_df = pd.DataFrame(columns=['LU Type ID',
                             'HU Type ID',
                             'Material Type ID',
                             'Allowed',
                             'Max Quan',
                             'Probability',
                             'Spot Time Probability',
                             'Spot Time Sigma',
                             'Spot Time Mean'])

In [23]:

all_lu_hu_mat_combs = set(product(lu_df["LU Type ID"].unique(),hu_type_df["HU Type ID"].unique(),material_df["Material Type ID"].unique()))
missing_combs = all_lu_hu_mat_combs - set(spottimes_df[["LU Type ID","HU Type ID","Material Type ID"]].to_records(index=False).tolist())
spottimes_df = spottimes_df[['LU Type ID',
                             'HU Type ID',
                             'Material Type ID',
                             'Allowed',
                             'Max Quan',
                             'Probability',
                             'Spot Time Probability',
                             'Spot Time Sigma',
                             'Spot Time Mean']].fillna(0)



In [27]:
if len(missing_combs)>0:
    missing_comb_df = pd.DataFrame.from_records(list(missing_combs),
                                                columns=['LU Type ID','HU Type ID','Material Type ID'])
#     missing_comb_df["Allowed"] = False
#     missing_comb_df["Max Quan"] = 0
#     missing_comb_df["Probability"] = 0
#     missing_comb_df["Spot Time Mean"] = 0
#     missing_comb_df["Spot Time Sigma"] = 0
#     missing_comb_df["Spot Time Probability"] = 0
    spottimes_df = pd.concat([spottimes_df,missing_comb_df])

In [28]:
spottimes_df

,LU Type ID,HU Type ID,Material Type ID,Allowed,Max Quan,Probability,Spot Time Probability,Spot Time Sigma,Spot Time Mean
0,2,3,1,NaN,NaN,NaN,NaN,NaN,NaN
1,9,1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,6,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,10,4,0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
115,3,0,1,NaN,NaN,NaN,NaN,NaN,NaN
116,6,4,1,NaN,NaN,NaN,NaN,NaN,NaN
117,7,2,1,NaN,NaN,NaN,NaN,NaN,NaN
118,11,2,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# write to Excel
book = load_workbook(fileName)
with pd.ExcelWriter(fileName, engine='openpyxl') as writer:
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    # sheet = book["General"]
    # sheet['P2'] = totalDur
    
    sheet = book["Random Delays"]
    sheet.delete_cols(1,5)
    randomDelaysDF = pd.concat([huRandomDelaysCombined, luRandomDelaysCombined, dumpDelay])
    randomDelaysDF.to_excel(writer, sheet_name='Random Delays', header=True, index=False, startrow=0, startcol=0)

    # sheet = book["LU PM"]
    # sheet.delete_cols(1,5)
    # lu_pm_df.to_excel(writer, sheet_name='LU PM', header=True, index=False, startrow=0, startcol=0)

    sheet = book["LU TBM"]
    sheet.delete_cols(1,5)    
    move_df.to_excel(writer, sheet_name='LU TBM', header=True, index=False, startrow=0, startcol=0)

    # sheet = book["LU Schedule"]
    # sheet.delete_cols(1,5)
    # ld_ll_df.to_excel(writer, sheet_name='LU Schedule', header=True, index=False, startrow=0, startcol=0)

    # hu_count_df.to_excel(writer, sheet_name='HU Types', header=True, index=False, startrow=0, startcol=5)

    # sheet = book["Material Types"]
    # sheet.delete_cols(1,5)
    # material_df.to_excel(writer, sheet_name='Material Types', header=True, index=True, startrow=0, startcol=0)

    sheet = book["HU-LU-Material"]
    sheet.delete_cols(1,5)
    spottimes_df.to_excel(writer, sheet_name='HU-LU-Material', header=True, index=False, startrow=0, startcol=0)

    sheet = book["Pass Counts"]
    sheet.delete_cols(1,5)    
    pc_df.to_excel(writer, sheet_name='Pass Counts', header=True, index=False, startrow=0, startcol=0)
    
    sheet = book["Pass Quan"]
    sheet.delete_cols(1,5)    
    pq_df.to_excel(writer, sheet_name='Pass Quan', header=True, index=False, startrow=0, startcol=0)
    
    sheet = book["Pass Time"]
    sheet.delete_cols(1,5)    
    pt_df.to_excel(writer, sheet_name='Pass Time', header=True, index=False, startrow=0, startcol=0)

    # sheet = book["Load Locations"]
    # sheet.delete_cols(1,5)
    # sheet.insert_cols(1,5)
    # loadlocations_df.to_excel(writer, sheet_name='Load Locations', header=True, index=False, startrow=0, startcol=0)
    
    # sheet = book["LoadLoc-Material"]
    # sheet.delete_cols(1,2)
    # sheet.insert_cols(1,2)
    # ll_material_df[['Load Loc ID','Material Type ID']].to_excel(writer, sheet_name='LoadLoc-Material', header=True, index=False, startrow=0, startcol=0)    
    # sheet.delete_cols(4,6)
    # ll_material_df[['Primary Dump Loc ID','Secondary Dump Loc ID','Is Stripping']].to_excel(writer, sheet_name='LoadLoc-Material', header=True, index=False, startrow=0, startcol=3)
    
    # sheet = book["Dump Locations"]
    # sheet.delete_cols(1,6)
    # sheet.insert_cols(1,6)
    # dumplocations_df.to_excel(writer, sheet_name='Dump Locations', header=True, index=False, startrow=0, startcol=0)

    # data_summary_df.to_excel(writer, sheet_name='Data Summary', header=True, index=True, startrow=0, startcol=0)
